# Multiple Factor Analysis (MFA)

In [1]:
#changement de dossier
import os
os.chdir("d:/Bureau/PythonProject/packages/scientisttools/data/")

#importation des données
import pandas as pd
url = "http://factominer.free.fr/factomethods/datasets/wine.txt"
wine = pd.read_table(url,sep="\t")
wine.head()

,Label,Soil,Odor.Intensity.before.shaking,Aroma.quality.before.shaking,Fruity.before.shaking,Flower.before.shaking,Spice.before.shaking,Visual.intensity,Nuance,Surface.feeling,...,Acidity,Astringency,Alcohol,Balance,Smooth,Bitterness,Intensity,Harmony,Overall.quality,Typical
2EL,Saumur,Env1,3.074,3.000,2.714,2.280,1.960,4.321,4.000,3.269,...,2.107,2.429,2.500,3.250,2.731,1.926,2.857,3.143,3.393,3.250
1CHA,Saumur,Env1,2.964,2.821,2.375,2.280,1.680,3.222,3.000,2.808,...,2.107,2.179,2.654,2.926,2.500,1.926,2.893,2.964,3.214,3.036
1FON,Bourgueuil,Env1,2.857,2.929,2.560,1.960,2.077,3.536,3.393,3.000,...,2.179,2.250,2.643,3.321,2.679,2.000,3.074,3.143,3.536,3.179
1VAU,Chinon,Env2,2.808,2.593,2.417,1.913,2.160,2.893,2.786,2.538,...,3.179,2.185,2.500,2.333,1.680,1.963,2.462,2.038,2.464,2.250
1DAM,Saumur,Reference,3.607,3.429,3.154,2.154,2.040,4.393,4.036,3.385,...,2.571,2.536,2.786,3.464,3.036,2.071,3.643,3.643,3.741,3.444


In [2]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21 entries, 2EL  to T2  
Data columns (total 31 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Label                          21 non-null     object 
 1   Soil                           21 non-null     object 
 2   Odor.Intensity.before.shaking  21 non-null     float64
 3   Aroma.quality.before.shaking   21 non-null     float64
 4   Fruity.before.shaking          21 non-null     float64
 5   Flower.before.shaking          21 non-null     float64
 6   Spice.before.shaking           21 non-null     float64
 7   Visual.intensity               21 non-null     float64
 8   Nuance                         21 non-null     float64
 9   Surface.feeling                21 non-null     float64
 10  Odor.Intensity                 21 non-null     float64
 11  Quality.of.odour               21 non-null     float64
 12  Fruity                         21 non-null     float

In [3]:
from prince import MFA

In [4]:
wine2 = pd.DataFrame(data=wine.values,
                     columns = pd.MultiIndex.from_tuples(
    [
        ("others","Label"),
        ("others","Soil"),
        ("before shaking","Odor.Intensity"),
        ("before shaking","Aroma.quality"),
        ("before shaking","Fruity"),
        ("before shaking","Flower"),
        ("before shaking","Spice"),
        ("vision","Visual.intensity"),
        ("vision","Nuance"),
        ("vision","Surface.feeling"),
        ("after shaking","Odor.intensity"),
        ("after shaking","Quality.of.odour"),
        ("after shaking","Fruity"),
        ("after shaking","Flower"),
        ("after shaking","Spice"),
        ("after shaking","Plante"),
        ("after shaking","Phenolic"),
        ("after shaking","Aroma.intensity"),
        ("after shaking","Aroma.persistency"),
        ("after shaking","Aroma.quality"),
        ("gustation","Attack.intensity"),
        ("gustation","Acidity"),
        ("gustation","Astringency"),
        ("gustation","Alcohol"),
        ("gustation","Balance"),
        ("gustation","Smooth"),
        ("gustation","Bitterness"),
        ("gustation","Intensity"),
        ("gustation","Harmony"),
        ("overall judgement","Overall.quality"),
        ("overall judgement","Typical")
    ]
))

In [5]:
groups = wine2.columns.levels[0].drop(["others","overall judgement"]).tolist()
groups

['after shaking', 'before shaking', 'gustation', 'vision']

In [6]:
wine2 = wine2[groups].astype("float")

In [7]:
import prince
mfa = prince.MFA(
    n_components=6,
    n_iter=3,
    copy=True,
    check_input=True,
    engine='sklearn',
    random_state=42
)
mfa = mfa.fit(wine2, groups=groups)

In [8]:
mfa.eigenvalues_summary

,eigenvalue,% of variance,% of variance (cumulative)
component,,,
0,3.462,55.05%,55.05%
1,1.367,21.73%,76.78%
2,0.615,9.79%,86.56%
3,0.372,5.92%,92.48%
4,0.270,4.30%,96.78%
5,0.202,3.22%,100.00%


In [9]:
mfa.row_coordinates(wine2).head(6)

,0,1,2,3,4,5
0,-0.238874,-0.796677,0.935737,-0.524407,0.351492,-0.462977
1,2.044793,-1.383315,1.513530,-0.729589,-0.071290,-0.615520
2,1.220141,-0.459020,0.062333,1.036356,-0.717976,-0.450136
3,4.381299,0.994551,-0.033460,-0.310046,-0.477621,0.966686
4,-2.695771,-0.120330,-0.689965,-0.830386,-0.816247,-0.290338
5,-0.868637,-0.326270,0.391083,1.274204,-0.070273,0.278612


In [10]:
import numpy as np
np.square(mfa.svd_.s)

array([3.46195044, 1.36676827, 0.61542908, 0.37219967, 0.27038248,
       0.20240331])

In [11]:
mfa.svd_.s

array([1.86063173, 1.16908865, 0.78449288, 0.61008169, 0.51998316,
       0.44989255])

In [12]:
mfa["after shaking"].eigenvalues_

array([4.70074947, 2.48304844, 1.04639148, 0.76548316, 0.49589369,
       0.19981859])

In [13]:
mfa.group_row_coordinates(wine2)

group     after shaking                                                    \
component             0         1         2         3         4         5   
0             -0.357349 -1.350813  0.731069 -1.118806  0.474202  0.178280   
1              2.119630 -1.248869  1.933167 -0.070483 -1.610003 -2.347666   
2              1.337619 -0.757873 -0.180294  2.248843 -2.818222 -2.245000   
3              4.734360  2.291265  0.785343  0.624816  0.696412 -1.552097   
4             -2.980065 -1.467456 -0.286561 -0.607499 -0.550167  0.919244   
5             -0.716090 -1.917507  0.000329  1.253489 -1.531911  0.336582   
6             -1.928137 -0.953143 -0.556584  0.655427 -1.229819  1.105040   
7             -0.474444  2.332716  2.826376 -2.129736  1.574820  2.278968   
8             -0.398505 -0.812036 -1.268089  0.719803  0.012008  0.036227   
9              1.037181  0.242903 -0.742514 -0.222915  2.286775  0.623097   
10            -1.052454  0.100972 -0.195318 -0.557537  0.466322  0.384811   
11            -0.821375  0.502286 -1.061376 -0.416650  0.951465  0.342603   
12            -1.898522 -0.769542  0.278028 -0.905149 -0.137966  0.851477   
13            -1.079296 -0.408037 -0.545191  0.707340 -0.269144  0.931473   
14            -1.225418 -1.267450  0.050132  0.823869 -1.155862  0.559449   
15            -0.056912 -1.882632 -0.613236  0.639505 -1.395855 -0.657644   
16            -2.939797 -0.820955 -0.623954 -1.038687  0.934734  2.353583   
17             0.159385 -1.235416  0.009459 -0.339682 -0.426678 -1.029416   
18             5.230552  0.885286 -0.903714  2.577184 -0.310514 -3.090095   
19             0.414535  4.299930 -1.125990 -1.559351  2.891111 -0.469236   
20             0.895104  4.236369  1.488919 -1.283781  1.148293  0.490320   

group     before shaking                                ... gustation  \
component              0         1         2         3  ...         2   
0              -0.407263 -1.279499  2.458383 -1.725928  ...  0.260639   
1               1.757936 -2.756439  4.961487 -0.348599  ... -0.194408   
2               1.792750  0.246577  0.579271  2.694108  ...  0.185176   
3               3.860684  1.108277  2.445325  4.674353  ... -2.503975   
4              -4.362643  0.893450 -3.308917 -4.833331  ...  0.511548   
5               0.766697 -0.234385  0.570286  1.582486  ...  0.528010   
6              -1.959197 -0.005639 -1.611425 -2.020836  ...  0.337252   
7               1.045222  0.848753  3.829785  0.410971  ...  0.570149   
8               1.491293  0.169234 -0.894931  2.302792  ...  0.224801   
9               1.802860 -1.409849 -1.454126  2.437927  ... -0.397951   
10             -0.839267  0.780554 -2.381689  0.066797  ...  0.222456   
11             -1.221207  0.556526 -1.215481 -1.443142  ...  0.320140   
12             -1.805001 -2.050432  0.136252 -2.956631  ...  0.768438   
13             -0.382491 -1.561365  0.069480 -1.019272  ...  1.249692   
14             -1.221282 -0.962439  0.075538 -1.824925  ...  0.402171   
15             -0.485267 -0.383653 -1.893250  0.410601  ...  0.233641   
16             -1.983130 -3.333157 -1.137079 -3.433197  ... -0.285957   
17              0.652676 -0.482734 -0.407995  0.507610  ... -0.254713   
18              3.131801 -1.616568  0.273806  3.791248  ... -1.779352   
19             -1.540887  5.136213 -3.337473  0.100947  ... -0.535396   
20             -0.094284  6.336575  2.242755  0.626022  ...  0.137638   

group                                      vision                      \
component         3         4         5         0         1         2   
0         -0.003733  0.648805 -1.951413 -1.037884  0.365596  0.292855   
1         -0.571160  0.774916 -1.739015  2.834983 -0.879381 -0.646124   
2          0.174855  0.281047 -1.140750  1.431321 -0.450993 -0.334820   
3         -3.774998 -0.953447  1.983051  4.232797 -1.233546 -0.860531   
4          1.139648 -1.001605  1.173087 -1.480013  0.449520  0.324071   
5          0.927021 -0.203002  0.6472

In [14]:
pd.DataFrame(mfa.svd_.U)

,0,1,2,3,4,5
0,-0.028015,-0.148705,0.260288,-0.187573,0.147508,-0.224564
1,0.239817,-0.258205,0.421010,-0.260964,-0.029918,-0.298555
2,0.143100,-0.085679,0.017339,0.370690,-0.301308,-0.218336
3,0.513846,0.185639,-0.009307,-0.110899,-0.200440,0.468885
4,-0.316164,-0.022460,-0.191924,-0.297018,-0.342549,-0.140827
5,-0.101875,-0.060900,0.108785,0.455765,-0.029491,0.135140
6,-0.182154,-0.052193,-0.115044,0.000327,-0.350357,0.142896
7,-0.015140,0.147209,0.516784,-0.060498,0.150468,0.224082
8,0.007737,-0.047139,-0.127637,0.334153,0.048419,0.159275
9,0.140974,-0.069916,-0.199092,0.075238,0.537906,0.195065


In [15]:
mfa.svd_.V[:5,:5]

array([[-0.15545012, -0.19606388, -0.1817978 , -0.04262386, -0.07088844],
       [ 0.22705312, -0.16163181, -0.13135482, -0.22598003,  0.27261812],
       [ 0.12573779, -0.12978475, -0.29050797,  0.37420198, -0.00379976],
       [-0.23267893,  0.02147026, -0.04179984, -0.10643228,  0.13629934],
       [ 0.04895236,  0.13716452, -0.00655122, -0.2783855 , -0.09243616]])

In [16]:
mfa.plot(wine2)

alt.Chart(...)

: 